<P><strong> <span style="color: rgb(192, 68, 238);font-size: 26px;"> Generate Test Data and Calculate Power Matrix </strong></p>

<p><span style="color: rgb(83, 77, 243);"> In this notebook, we generate test data and calculate the power matrix for it. Test data is in fact some random locations for sites in Tehran. We grid the map and calculate the power of sectors received in grid centers and form the power matrix accordingly.
    </span></p>

In [1]:
import pandas as pd
import utm
import folium
import numpy as np
import random
from numpy import savetxt
from numpy import loadtxt
import math
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from colorama import Fore
from shapely.geometry import Polygon
from shapely.geometry import Point
from folium.features import DivIcon

from math import cos, sin , radians
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'

pd.options.plotting.backend = "plotly"

In [2]:
tehran_bbox = [35.5591, 51.0216, 35.8878, 51.6723]  # [min_lat, min_lon, max_lat, max_lon]

site_count = 26

<p><span style="color: rgb(83, 77, 243);">
We generate dataframe of sites.
 </span></p> 

In [3]:
# site_data = []

# for i in range(site_count):
#     lat = random.uniform(tehran_bbox[0], tehran_bbox[2])
#     long = random.uniform(tehran_bbox[1], tehran_bbox[3])
    
#     site_data.append([i,lat,long,40,140,250])

# df = pd.DataFrame(data=site_data, columns=['site_ID', 'lat', 'long', 'sec1_ang', 'sec2_ang', 'sec3_ang'])    

<p><span style="color: rgb(83, 77, 243);">
Saving sites dataframe (only the first time after data generation):
 </span></p> 

In [29]:
example_number = 1
# df.to_csv('random_example_{}_sites_lat_long_angle.csv'.format(example_number))

<p><span style="color: rgb(83, 77, 243);">
Laoding the saved data:
 </span></p> 

In [4]:
to_load_example_number = 1
df  = pd.read_csv('random_example_{}_sites_lat_long_angle.csv'.format(to_load_example_number))

In [5]:
df

,Unnamed: 0,site_ID,lat,long,sec1_ang,sec2_ang,sec3_ang
0,0,0,35.696925,51.608606,40,140,250
1,1,1,35.767535,51.302619,40,140,250
2,2,2,35.637088,51.427273,40,140,250
3,3,3,35.866706,51.392771,40,140,250
4,4,4,35.633080,51.172013,40,140,250
5,5,5,35.696897,51.130935,40,140,250
6,6,6,35.567121,51.371031,40,140,250
7,7,7,35.673432,51.032829,40,140,250
8,8,8,35.595892,51.283268,40,140,250
9,9,9,35.668678,51.213177,40,140,250


<P><strong> <span style="color: rgb(24, 230, 245);font-size: 20px;"> Geographical data </strong></p>

<p><span style="color: rgb(83, 77, 243);">
Lets see some geographical data like distances between grid centers. 
 </span></p> 

In [6]:
df['utm_x']=df.apply(lambda x:utm.from_latlon(x['lat'], x['long'])[0],axis=1) # note: Axis along which the function is applied: 1 or ‘columns’: apply function to each row.

df['utm_y']=df.apply(lambda x:utm.from_latlon(x['lat'], x['long'])[1],axis=1)

print(Fore.MAGENTA + "horizontal spread of sites: {} km \nvertical spread of sites: {} km".format(int((df.utm_x.max() - df.utm_x.min())/1000), int((df.utm_y.max() - df.utm_y.min())/1000)))  

horizontal spread of sites: 55 km 
vertical spread of sites: 33 km


<p><span style="color: rgb(83, 77, 243);">
We set the number of grids in horizontal and vertical axis.
 </span></p> 

In [7]:
n_squares_in_lat=7
n_squares_in_long=9

delta_lat=(df.lat.max()-df.lat.min())/n_squares_in_lat
delta_long=(df.long.max()-df.long.min())/n_squares_in_long

In [8]:
map_lat_min=df.lat.min()-delta_lat
map_lat_max=df.lat.max()+delta_lat
map_long_min=df.long.min()-delta_long
map_long_max=df.long.max()+delta_long

print(Fore.MAGENTA + "min lat in map is: {}, max lat in map is: {}, the spread is: {}".format(df.lat.min(),df.lat.max(),df.lat.max()-df.lat.min()))
print("min long in map is: {}, max long in map is: {}, the spread is: {}".format(df.long.min(),df.long.max(),df.long.max()-df.long.min()))
print("delta in lat direction is: {}".format(delta_lat))
print("delta in long direction is: {}".format(delta_long))


min lat in map is: 35.56712096461399, max lat in map is: 35.866706461359406, the spread is: 0.299585496745415
min long in map is: 51.02615556848854, max long in map is: 51.63708115661623, the spread is: 0.6109255881276923
delta in lat direction is: 0.04279792810648786
delta in long direction is: 0.06788062090307692


<p><span style="color: rgb(83, 77, 243);">
We can see the vertical and horizontal dimensions of each grid in km. We see this for a grid in north east of Tehran map, but since grids are small comapred to Tehran map, the dimensions of grid in other locations are the same.
 </span></p> 

In [9]:
print(Fore.MAGENTA + "length of delta_long at north east of Tehran:\nhorizontal:{} km ***** vertical: {} km".format(
    int(np.abs(utm.from_latlon(map_lat_max,map_long_max)[0]-utm.from_latlon(map_lat_max,map_long_max+delta_long)[0])/1000),
    int(np.abs(utm.from_latlon(map_lat_max,map_long_max)[1]-utm.from_latlon(map_lat_max,map_long_max+delta_long)[1])/1000)))

length of delta_long at north east of Tehran:
horizontal:6 km ***** vertical: 0 km


In [10]:
print(Fore.MAGENTA + "length of delta_lat at north east of Tehran:\nhorizontal:{} km ***** vertical: {} km".format(
    int(np.abs(utm.from_latlon(map_lat_max,map_long_max)[0]-utm.from_latlon(map_lat_max+delta_lat,map_long_max)[0])/1000),
    int(np.abs(utm.from_latlon(map_lat_max,map_long_max)[1]-utm.from_latlon(map_lat_max+delta_lat,map_long_max)[1])/1000)))

length of delta_lat at north east of Tehran:
horizontal:0 km ***** vertical: 4 km


<P><strong> <span style="color: rgb(24, 230, 245);font-size: 20px;"> Griding </strong></p>

<p><span style="color: rgb(83, 77, 243);">
We make list and dictionary of grid centers.
 </span></p> 

In [11]:
grid_centers=[]
grid_centers_dict={}
grid_id = 0
for lat in np.arange(map_lat_min,map_lat_max,delta_lat): # step: delta_lat
    for long in np.arange(map_long_min,map_long_max,delta_long): # step: delta_long
        
        if lat+(delta_lat/2)<map_lat_max and long+(delta_long/2) < map_long_max:
            grid_centers.append([lat+(delta_lat/2),long+(delta_long/2)])
            grid_centers_dict[grid_id]=[lat+(delta_lat/2),long+(delta_long/2)]
            grid_id += 1

In [12]:
print(Fore.MAGENTA + "The number of grig centers: {}".format(len(grid_centers)))

The number of grig centers: 99


<p><span style="color: rgb(83, 77, 243);">
Now we see sites and sectors on map. The pink squares show the site ID. Sectors are numbered in counter clockwise order (0,1,2). 
 </span></p> 

In [42]:
tehran_map = folium.Map(location=[(map_lat_min+map_lat_max)/2, (map_long_min+map_long_max)/2],crs='EPSG3857', zoom_start=10)
  
folium.PolyLine([(map_lat_min,map_long_min),(map_lat_max,map_long_min)], color="gray", weight=2.5, opacity=1).add_to(tehran_map)
folium.PolyLine([(map_lat_min,map_long_min),(map_lat_min,map_long_max)], color="gray", weight=2.5, opacity=1).add_to(tehran_map)
folium.PolyLine([(map_lat_min,map_long_max),(map_lat_max,map_long_max)], color="gray", weight=2.5, opacity=1).add_to(tehran_map)
folium.PolyLine([(map_lat_max,map_long_min),(map_lat_max,map_long_max)], color="gray", weight=2.5, opacity=1).add_to(tehran_map)

size = 0.02

theta1 = 10
theta2 = 70
theta3 = 110
theta4 = 170
theta5 = 220
theta6 = 280
# These degrees can be different for the three sectors.

for i,row in df.iterrows():
    center = [row['lat'],row['long']]
    vertices_1 = [
    [center[0], center[1]],
    [center[0] + size * math.sin(math.radians(theta1)), center[1] + size * math.cos(math.radians(theta1))],
    [center[0] + size * math.sin(math.radians(theta2)), center[1] + size * math.cos(math.radians(theta2))]]
    vertices_2 = [
    [center[0], center[1]],
    [center[0] + size * math.sin(math.radians(theta3)), center[1] + size * math.cos(math.radians(theta3))],
    [center[0] + size * math.sin(math.radians(theta4)), center[1] + size * math.cos(math.radians(theta4))]]
    vertices_3 = [
    [center[0], center[1]],
    [center[0] + size * math.sin(math.radians(theta5)), center[1] + size * math.cos(math.radians(theta5))],
    [center[0] + size * math.sin(math.radians(theta6)), center[1] + size * math.cos(math.radians(theta6))]
        ]
    polygon_1 = folium.Polygon(locations=vertices_1, color='red', fill=True, fill_opacity=0.7)
    polygon_2 = folium.Polygon(locations=vertices_2, color='blue', fill=True, fill_opacity=0.7)
    polygon_3 = folium.Polygon(locations=vertices_3, color='green', fill=True, fill_opacity=0.7)
    
    length = 0.007
    square_vertices = [[center[0]-length, center[1]-length],[center[0]-length, center[1]+length],[center[0]+length, center[1]+length],[center[0]+length, center[1]-length]]
    polygon_4 = folium.Polygon(locations=square_vertices, color='magenta', fill=True, fill_opacity=0.7)
    
    
    triangle_center_1 = [sum(p[0] for p in vertices_1) / len(vertices_1), sum(p[1] for p in vertices_1) / len(vertices_1)]
    triangle_center_2 = [sum(p[0] for p in vertices_2) / len(vertices_2), sum(p[1] for p in vertices_2) / len(vertices_2)]
    triangle_center_3 = [sum(p[0] for p in vertices_3) / len(vertices_3), sum(p[1] for p in vertices_3) / len(vertices_3)]
    icon_html_1 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">0</div>'
    icon_html_2 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">1</div>'
    icon_html_3 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">2</div>'
    icon_html_4 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">{}</div>'.format(i)
    
    icon_1 = folium.DivIcon(html=icon_html_1)
    icon_2 = folium.DivIcon(html=icon_html_2)
    icon_3 = folium.DivIcon(html=icon_html_3)
    icon_4 = folium.DivIcon(html=icon_html_4)
    
    polygon_1.add_to(tehran_map)
    polygon_2.add_to(tehran_map)
    polygon_3.add_to(tehran_map)
    polygon_4.add_to(tehran_map)
    
    folium.Marker(location=triangle_center_1, icon=icon_1).add_to(tehran_map)
    folium.Marker(location=triangle_center_2, icon=icon_2).add_to(tehran_map)
    folium.Marker(location=triangle_center_3, icon=icon_3).add_to(tehran_map)
    folium.Marker(location=center, icon=icon_4).add_to(tehran_map)
    
# for _,row in df.iterrows():
#     popup_html = '<p>site {}</p><p>lat: {}</p><p>long: {}</p>'.format(int(row['site_ID']),row['lat'],row['long'])
#     folium.CircleMarker(location=[row['lat'], row['long']], radius=5, color='purple', fill=True, fill_color='purple',popup=folium.Popup(popup_html)).add_to(tehran_map)
      

for key, value in grid_centers_dict.items():
    popup_html = '<p>***grid {}***</p><p>lat: {}</p><p>long: {}</p>'.format(key,round(value[0],5),round(value[1],5))
    folium.CircleMarker(location=[value[0], value[1]], radius=4, color='#4CBB17', fill=True, fill_color='#4CBB17',popup=folium.Popup(popup_html,min_width=100,max_width=100)).add_to(tehran_map)

    
tehran_map

<P><strong> <span style="color: rgb(24, 230, 245);font-size: 20px;"> Power Matrix </strong></p>

<p><span style="color: rgb(83, 77, 243);">
The following function determines if a grid receives signal from a sector or not. It returns true or false. 
 </span></p> 
 <p><span style="color: rgb(83, 77, 243);">
Angle should be standard gradian (zero degree is at east, the direction is counter clockwise.)
 </span></p> 

In [14]:
def check_angular_coverage(grid_to_sector_angle, center_of_sector_angle, coverage_angle):
    
    theta_min = center_of_sector_angle - coverage_angle/2
    theta_max = center_of_sector_angle + coverage_angle/2
    
    if theta_min<0:
        if grid_to_sector_angle >= theta_min +360 or grid_to_sector_angle<=theta_max:
            return True
        else:
            return False
    elif theta_max>360:
        if grid_to_sector_angle >= theta_min or grid_to_sector_angle<=theta_max-360:
            return True
        else:
            return True
    elif grid_to_sector_angle >= theta_min and grid_to_sector_angle<=theta_max:
        return True
    else:
        return False

In [15]:
# test
check_angular_coverage(350,10,60)

True

<p><span style="color: rgb(83, 77, 243);">
The distance_threshold parameter specifies the radius of an imaginary circle around a site. This circle is the boundary of the region that each site covers.
 </span></p> 
 
 <p><span style="color: rgb(83, 77, 243);">
distance_threshold should not be smaller than dimensions of a grid, otherwise each sector may transmit signal to at most one grid center. Here, distance_threshold is set to approximately twice the length of each grid square.
 </span></p> 

In [16]:
distance_threshold = 13 # in km

<p><span style="color: rgb(83, 77, 243);">
The following function calculates the received power from a sector in a grid center. If the sector is further than distance_threshold from grid center, the received power is zero. If the sector is within distance_threshold from grid center, the angle from grid center to sector is checked to see if the grid center is covered by the sector. If covered, the power can be calculated by the formula 40/d<sup>2</sup>, where d is the distance between grid center and the sector.
 </span></p> 
 
 <p><span style="color: rgb(83, 77, 243);">
Again, the angle is in radian. 
 </span></p> 
 
 <p><span style="color: rgb(83, 77, 243);">
Note that the coverage angle of all sectors is set to 60.
 </span></p> 

In [17]:
def power_of_sector_in_grid(grid_center_lat, grid_center_long, sector_lat,sector_long,sector_angle):
    
    sector_x , sector_y ,_ ,_ = utm.from_latlon(sector_lat, sector_long)

    covered_gird_centers = []
    received_power_in_covered_grid_centers = []

#     sector_angle=90-float(sector_angle)
#     if sector_angle<0:
#         sector_angle=sector_angle+360
#     if sector_angle>360:
#         sector_angle=sector_angle-360

    grid_center_x , grid_center_y, _, _ = utm.from_latlon(grid_center_lat,grid_center_long)

    distance=geodesic((grid_center_lat,grid_center_long), (sector_lat, sector_long)).km
    
    if distance>distance_threshold:
        return 0

    grid_to_sector_angle = math.degrees(math.atan2(grid_center_y - sector_y, grid_center_x - sector_x))
    if grid_to_sector_angle<0:
        grid_to_sector_angle = grid_to_sector_angle + 360
    
#     print(grid_to_sector_angle)
    
    if check_angular_coverage(grid_to_sector_angle, sector_angle, 60):
        return 40/(distance**2)
    else:
        return 0                                              

<p><span style="color: rgb(83, 77, 243);">
Using the above function, we can form the power matrix. Element (i,j) of this matrix specifies the received power of sector j in grid center i.
 </span></p> 

In [18]:
power_matrix = np.zeros((len(grid_centers),len(df)*3),dtype='float16')

for i in range(len(grid_centers)):
    for j in range(len(df)):
        power_matrix[i,3*j] = power_of_sector_in_grid(grid_centers[i][0], grid_centers[i][1], df.iloc[j]['lat'], df.iloc[j]['long'],40)
        power_matrix[i,3*j+1] = power_of_sector_in_grid(grid_centers[i][0], grid_centers[i][1], df.iloc[j]['lat'], df.iloc[j]['long'],140)
        power_matrix[i,3*j+2] = power_of_sector_in_grid(grid_centers[i][0], grid_centers[i][1], df.iloc[j]['lat'], df.iloc[j]['long'],250)
        

<p><span style="color: rgb(83, 77, 243);">
In order to verify the power matrix, we need to know the sectors that cover each grid center. In the following dictionary whose keys are grid centers, we list the sectors that cover each grid center.
 </span></p> 

In [47]:
close_aligned_sectors_for_grids_dict = {key:[(b//3,b%3) for b in range(len(df)*3) if power_matrix[int(key),b]>0] for key in range(len(grid_centers))}

In [23]:
# close_aligned_sectors_for_grids_dict

<p><span style="color: rgb(83, 77, 243);">
Lets verify the power matrix with this map. By clicking on each grid center, we can see the (site,sector) that transmits signal to that grid center.
 </span></p> 

In [48]:
tehran_map = folium.Map(location=[(map_lat_min+map_lat_max)/2, (map_long_min+map_long_max)/2],crs='EPSG3857', zoom_start=10)
  
folium.PolyLine([(map_lat_min,map_long_min),(map_lat_max,map_long_min)], color="gray", weight=2.5, opacity=1).add_to(tehran_map)
folium.PolyLine([(map_lat_min,map_long_min),(map_lat_min,map_long_max)], color="gray", weight=2.5, opacity=1).add_to(tehran_map)
folium.PolyLine([(map_lat_min,map_long_max),(map_lat_max,map_long_max)], color="gray", weight=2.5, opacity=1).add_to(tehran_map)
folium.PolyLine([(map_lat_max,map_long_min),(map_lat_max,map_long_max)], color="gray", weight=2.5, opacity=1).add_to(tehran_map)

size = 0.02

theta1 = 10
theta2 = 70
theta3 = 110
theta4 = 170
theta5 = 220
theta6 = 280
# These degrees can be different for the three sectors.

for i,row in df.iterrows():
    center = [row['lat'],row['long']]
    vertices_1 = [
    [center[0], center[1]],
    [center[0] + size * math.sin(math.radians(theta1)), center[1] + size * math.cos(math.radians(theta1))],
    [center[0] + size * math.sin(math.radians(theta2)), center[1] + size * math.cos(math.radians(theta2))]]
    vertices_2 = [
    [center[0], center[1]],
    [center[0] + size * math.sin(math.radians(theta3)), center[1] + size * math.cos(math.radians(theta3))],
    [center[0] + size * math.sin(math.radians(theta4)), center[1] + size * math.cos(math.radians(theta4))]]
    vertices_3 = [
    [center[0], center[1]],
    [center[0] + size * math.sin(math.radians(theta5)), center[1] + size * math.cos(math.radians(theta5))],
    [center[0] + size * math.sin(math.radians(theta6)), center[1] + size * math.cos(math.radians(theta6))]
        ]
    polygon_1 = folium.Polygon(locations=vertices_1, color='red', fill=True, fill_opacity=0.7)
    polygon_2 = folium.Polygon(locations=vertices_2, color='blue', fill=True, fill_opacity=0.7)
    polygon_3 = folium.Polygon(locations=vertices_3, color='green', fill=True, fill_opacity=0.7)
    
    length = 0.007
    square_vertices = [[center[0]-length, center[1]-length],[center[0]-length, center[1]+length],[center[0]+length, center[1]+length],[center[0]+length, center[1]-length]]
    polygon_4 = folium.Polygon(locations=square_vertices, color='magenta', fill=True, fill_opacity=0.7)
    
    triangle_center_1 = [sum(p[0] for p in vertices_1) / len(vertices_1), sum(p[1] for p in vertices_1) / len(vertices_1)]
    triangle_center_2 = [sum(p[0] for p in vertices_2) / len(vertices_2), sum(p[1] for p in vertices_2) / len(vertices_2)]
    triangle_center_3 = [sum(p[0] for p in vertices_3) / len(vertices_3), sum(p[1] for p in vertices_3) / len(vertices_3)]
    icon_html_1 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">0</div>'
    icon_html_2 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">1</div>'
    icon_html_3 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">2</div>'
    icon_html_4 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">{}</div>'.format(i)
    
    icon_1 = folium.DivIcon(html=icon_html_1)
    icon_2 = folium.DivIcon(html=icon_html_2)
    icon_3 = folium.DivIcon(html=icon_html_3)
    icon_4 = folium.DivIcon(html=icon_html_4)
    
    polygon_1.add_to(tehran_map)
    polygon_2.add_to(tehran_map)
    polygon_3.add_to(tehran_map)
    polygon_4.add_to(tehran_map)
    
    folium.Marker(location=triangle_center_1, icon=icon_1).add_to(tehran_map)
    folium.Marker(location=triangle_center_2, icon=icon_2).add_to(tehran_map)
    folium.Marker(location=triangle_center_3, icon=icon_3).add_to(tehran_map)
    folium.Marker(location=center, icon=icon_4).add_to(tehran_map)
    
# for _,row in df.iterrows():
#     popup_html = '<p>site {}</p><p>lat: {}</p><p>long: {}</p>'.format(int(row['site_ID']),row['lat'],row['long'])
#     folium.CircleMarker(location=[row['lat'], row['long']], radius=5, color='purple', fill=True, fill_color='purple',popup=folium.Popup(popup_html)).add_to(tehran_map)
      

for key, value in grid_centers_dict.items():
    popup_html = '<p>***grid {}***</p>'.format(key,round(value[0],5),round(value[1],5))
    for x in close_aligned_sectors_for_grids_dict[key]:
        popup_html += '<p>{} </p>'.format(x)
    folium.CircleMarker(location=[value[0], value[1]], radius=4, color='#4CBB17', fill=True, fill_color='#4CBB17',popup=folium.Popup(popup_html,min_width=100,max_width=100)).add_to(tehran_map)

    
tehran_map

<p><span style="color: rgb(83, 77, 243);">
Saving the power matrix.
 </span></p> 

In [104]:
savetxt('random_example_{}_power_matrix.csv'.format(example_number), power_matrix, delimiter=',')
# loaded_power_matrix = loadtxt('power_matrix_example_3.csv', delimiter=',')

<P><strong> <span style="color: rgb(24, 230, 245);font-size: 20px;"> Inspecting the DOCPLEX solution </strong></p>

In [30]:
cplex_allocation_matrix = loadtxt('random_example_{}_docplex_solution.csv'.format(example_number), delimiter=',')

<p><span style="color: rgb(83, 77, 243);">
We translate the binary solution matrix to a vector that contains the PCI value of each sector.
 </span></p> 

In [31]:
pci_max_third = 3  # pci_max = 3*3+2 = 11
pci_max = pci_max_third*3 + 2
pci_vector = np.array(range(pci_max))
cplex_allocation_vector = cplex_allocation_matrix @ pci_vector

<p><span style="color: rgb(83, 77, 243);">
For each grid center, we make a list of sectors that transmit signal to the grid center, along with PCI value of that sector. 
 </span></p> 

In [32]:
pci_of_close_aligned_sectors_for_grids_dict = {key:['sector:({},{}),pci:{}'.format(b//3,b%3,int(cplex_allocation_vector[b])) for b in range(len(df)*3) if power_matrix[int(key),b]>0] for key in range(len(grid_centers))}
# pci_of_close_aligned_sectors_for_grids_dict

<p><span style="color: rgb(83, 77, 243);">
Now we can see the docplex solution on map. PCI values of sectors are shown on the triangles. By clicking on eah grid center, we can see the PCI values of sectors that transmit signal to that grid center. If there is any collision in a grid, we can see the same PCI values.
 </span></p> 

In [49]:
pci_tehran_map = folium.Map(location=[(map_lat_min+map_lat_max)/2, (map_long_min+map_long_max)/2],crs='EPSG3857', zoom_start=10)

size = 0.03

theta1 = 10
theta2 = 70
theta3 = 110
theta4 = 170
theta5 = 220
theta6 = 280


for i,row in df.iterrows():
    center = [row['lat'],row['long']]
    vertices_1 = [
    [center[0], center[1]],
    [center[0] + size * math.sin(math.radians(theta1)), center[1] + size * math.cos(math.radians(theta1))],
    [center[0] + size * math.sin(math.radians(theta2)), center[1] + size * math.cos(math.radians(theta2))]]
    vertices_2 = [
    [center[0], center[1]],
    [center[0] + size * math.sin(math.radians(theta3)), center[1] + size * math.cos(math.radians(theta3))],
    [center[0] + size * math.sin(math.radians(theta4)), center[1] + size * math.cos(math.radians(theta4))]]
    vertices_3 = [
    [center[0], center[1]],
    [center[0] + size * math.sin(math.radians(theta5)), center[1] + size * math.cos(math.radians(theta5))],
    [center[0] + size * math.sin(math.radians(theta6)), center[1] + size * math.cos(math.radians(theta6))]
        ]
    polygon_1 = folium.Polygon(locations=vertices_1, color='red', fill=True, fill_opacity=0.7)
    polygon_2 = folium.Polygon(locations=vertices_2, color='blue', fill=True, fill_opacity=0.7)
    polygon_3 = folium.Polygon(locations=vertices_3, color='green', fill=True, fill_opacity=0.7)
    
    length = 0.007
    square_vertices = [[center[0]-length, center[1]-length],[center[0]-length, center[1]+length],[center[0]+length, center[1]+length],[center[0]+length, center[1]-length]]
    polygon_4 = folium.Polygon(locations=square_vertices, color='magenta', fill=True, fill_opacity=0.7)
    
    triangle_center_1 = [sum(p[0] for p in vertices_1) / len(vertices_1), sum(p[1] for p in vertices_1) / len(vertices_1)]
    triangle_center_2 = [sum(p[0] for p in vertices_2) / len(vertices_2), sum(p[1] for p in vertices_2) / len(vertices_2)]
    triangle_center_3 = [sum(p[0] for p in vertices_3) / len(vertices_3), sum(p[1] for p in vertices_3) / len(vertices_3)]
    
    
    
    icon_html_1 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">{}</div>'.format(int(cplex_allocation_vector[3*i]))
    icon_html_2 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">{}</div>'.format(int(cplex_allocation_vector[3*i+1]))
    icon_html_3 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">{}</div>'.format(int(cplex_allocation_vector[3*i+2]))
    icon_html_4 = '<div style="font-size: 8pt; font-weight: bold; color: white; text-align: center;">{}</div>'.format(i)
    
    icon_1 = folium.DivIcon(html=icon_html_1)
    icon_2 = folium.DivIcon(html=icon_html_2)
    icon_3 = folium.DivIcon(html=icon_html_3)
    icon_4 = folium.DivIcon(html=icon_html_4)
    
    folium.CircleMarker(location=[row['lat'], row['long']], radius=6, color='black', fill=True, fill_color='black',popup=folium.Popup(' '.join(map(str,['site '+str(row['site_ID']),row['lat'],row['long']])))).add_to(pci_tehran_map)
    polygon_1.add_to(pci_tehran_map)
    polygon_2.add_to(pci_tehran_map)
    polygon_3.add_to(pci_tehran_map)
    polygon_4.add_to(pci_tehran_map)
    
    
    folium.Marker(location=triangle_center_1, icon=icon_1).add_to(pci_tehran_map)
    folium.Marker(location=triangle_center_2, icon=icon_2).add_to(pci_tehran_map)
    folium.Marker(location=triangle_center_3, icon=icon_3).add_to(pci_tehran_map)
    folium.Marker(location=center, icon=icon_4).add_to(pci_tehran_map)
    
    
    

for key, value in grid_centers_dict.items():
    popup_html = '<p>***grid {}***</p>'.format(key)
    for x in pci_of_close_aligned_sectors_for_grids_dict[key]:
        popup_html += '<p>{} </p>'.format(x)
    folium.CircleMarker(location=[value[0], value[1]], radius=4, color='#4CBB17', fill=True, fill_color='#4CBB17',popup=folium.Popup(popup_html,min_width=100,max_width=100)).add_to(pci_tehran_map)

    
    
pci_tehran_map

In [45]:
pci_tehran_map.save('./random_example_{}_PCI_solution.html'.format(example_number))